In [1]:
!pip install ktrain
!pip install transformers

     |████████████████████████████████| 25.3MB 133kB/s 
     |████████████████████████████████| 983kB 49.7MB/s 
     |████████████████████████████████| 266kB 48.1MB/s 
     |████████████████████████████████| 1.3MB 40.9MB/s 
     |████████████████████████████████| 1.1MB 49.2MB/s 
     |████████████████████████████████| 471kB 47.7MB/s 
     |████████████████████████████████| 890kB 46.5MB/s 
     |████████████████████████████████| 2.9MB 49.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.2-cp36-none-any.whl size=25276306 sha256=84360529a3e725656b269e977eaa5f41cb956c294f58b2a254b148a947a8b99a
  Stored in directory: /root/.cache/pip/wheels/fe/56/00/25444c352cc843e5c5daea0e9517a192878ae22c2c6b5f4573
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=65e4934997b9d4cf58e0d2e68e522b7218e42fdd1ebd4ae17f586e1ab2e94c30
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for syn

In [3]:
import ktrain
import random 
import pickle
import torch
import time
import transformers
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.keras.utils as ku
import torch.nn as nn

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from ktrain import text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModel, BertTokenizerFast
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive


In [5]:
device = torch.device("cuda")

In [6]:
%matplotlib inline

In [7]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
!pwd


/content


In [9]:
cd /content/drive/MyDrive/assignmnet data

/content/drive/MyDrive/assignmnet data


In [10]:
pwd

'/content/drive/MyDrive/assignmnet data'

In [11]:
!git clone https://github.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation.git

Cloning into 'SemEval2020-Task4-Commonsense-Validation-and-Explanation'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 88 (delta 36), reused 64 (delta 19), pack-reused 0
Unpacking objects: 100% (88/88), done.


In [12]:

trf = '/content/drive/MyDrive/assignmnet data/SemEval2020-Task4-Commonsense-Validation-and-Explanation/ALL data/train.csv'
train_data = pd.read_csv(trf)
tf = '/content/drive/MyDrive/assignmnet data/SemEval2020-Task4-Commonsense-Validation-and-Explanation/ALL data/test.csv'
test_data = pd.read_csv(tf)
vf = '/content/drive/MyDrive/assignmnet data/SemEval2020-Task4-Commonsense-Validation-and-Explanation/ALL data/dev.csv'
valid_data = pd.read_csv(vf)

In [13]:
train_data

,Correct Statement,Incorrect Statement,Right Reason1,Confusing Reason1,Confusing Reason2,Right Reason2,Right Reason3
0,He poured milk on his cereal.,He poured orange juice on his cereal.,Orange juice doesn't taste good on cereal.,Orange juice is usually bright orange.,Orange juice is sticky if you spill it on the ...,Orange juice is poured in a glass.,Orange juice does not taste good on cereal.
1,He drinks milk.,He drinks apple.,Apple can not be drunk,Apple juice are very tasty and milk too,Apple cannot eat a human,An apple is a whole food and unable to be drun...,He eats an apple
2,Jeff ran a mile today,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km.",This is not physically possible to achieve,"No way can someone run 100,000 miles in a day"
3,A mosquito stings me,I sting a mosquito,A human has not stings,A human is a mammal,A human is omnivorous,I do not have a stinger.,"mosquitos sting people, not the other way around"
4,A niece is a person.,A giraffe is a person.,A giraffe is not a human being.,Giraffes can drink water from a lake.,.Giraffes usually eat leaves.,"A giraffe is an animal, not human.",A giraffe is an animal.
...,...,...,...,...,...,...,...
9995,Mark ate a big sweet cherry pie,Mark ate a big bitter cherry pie,a cherry pie should be sweet,Mark is bad at making cherry pie,a cherry pie should be big,cherry pies are sweet,There's no bitter cherry pie.
9996,Gloria wears a hat on her head,Gloria wears a cat on her head,a cat cannot be worn on a person's head,a hat cannot be worn on a cat's head,the cat is too heavy to be worn on her head,a cat cannot be worn as a cap,"A cat is not a cap to wear on head, its an animal"
9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...,a barbershop usually don't provide the service...,a barbershop usually repairs computers instead...,the barbershop lacked the necessary tools to r...,Barbershop do not repair glasses.,BARBERSHOP IS ONLY USE FOR HAIR CUT NOR FOR GL...
9998,Reilly is sleeping on the couch,Reilly is sleeping on the window,a person cannot sleep on a window,the window is open and a person cannot lay on it,the window is too cold to sleep on it,Because she went in the train,Reilly is on the window sleeping


In [14]:
test_data

,Correct Statement,Incorrect Statement,Right Reason1,Confusing Reason1,Confusing Reason2,Right Reason2,Right Reason3
0,when it rains humidity forms,when it is hot humidity forms,hotness will evaporate water,Humidity is a measure of moisture in the atmos...,Laundry will not be dry because of the humidity.,Laundry will not be dry because of the humidity.,"Water makes humidity, not temperature."
1,Grizzly bears love honey.,Grizzly bears hate honey.,Honey is good for grizzly bear's growth,Grizzly bears have been observed taking stings...,Bees cannot eat grizzly bears,Bees cannot eat grizzly bears,Grizzly bears have been observed in the wild s...
2,shrimps can live underwater,dogs can live underwater,dogs haven't organs to breath underwater,dogs can swim in the water,Some dogs hate the water,Some dogs hate the water,"Dogs need air to breath, so they cannot live u..."
3,ice hockey is a sport,ice hockey is a financial institution,There are no relationships between ice hockey ...,Children's playing ice hockey requires financi...,Playing ice hockey well can bring you money,Playing ice hockey well can bring you money,Ice hockey is a sport played between two teams...
4,She inserts the key in the lock.,She inserts her finger in the lock,Her finger is bigger than lock's eye,finger is soft while the lock is hard,finger cannot be used as a lock,finger cannot be used as a lock,A finger wouldnt fit into a lock.
...,...,...,...,...,...,...,...
995,Soccer balls are round.,American Football balls are round.,American Football balls are not round. They ha...,You can play soccer and football using your sh...,Soccer and American Football balls are not mil...,Soccer and American Football balls are not mil...,American footballs have a unique shape and are...
996,The zebra ate its lunch.,He ate an entire zebra for lunch.,People can not eat an entire zebra.,Zebras don't eat people,Not everyone has eaten zebra,Not everyone has eaten zebra,Zebra is a wild animal and cannot be eaten .
997,I sliced the strong meat with my Kitchen Knife.,I sliced the strong meat with my Kitchen spoon.,Spoons are not used for slicing food like a kn...,A spoon can hold some liquid while a knife can...,A spoon is not sharp and can be used by babies...,A spoon is not sharp and can be used by babies...,Spoons are not sharp enough to cut meat.
998,today my eye is very clear to see the world,today my hand is very clear to see the world,People use eyes but not hands to see the world,hands are not used for eating food,people should clean their hands and eyes,people should clean their hands and eyes,No-one sees by hand.


In [16]:
valid_data

,Correct Statement,Incorrect Statement,Right Reason1,Confusing Reason1,Confusing Reason2,Right Reason2,Right Reason3
0,A chainsaw is used to cut trees,A chainsaw is used to cut diamonds,Cutting diamonds with a chainsaw would break it,chainsaw are produced from the factory but dia...,Diamonds don't grow on trees,Diamonds don't grow on trees,chainsaws arent used for diamonds
1,The closet got hangers,The hangers bought the closet,Hangers are inanimate things which cannot buy ...,A hanger is something you put your clothes on,The hangers cannot eat,The hangers cannot eat,Hangers dont have money.
2,A pillow is soft.,Feathers in a pillow are hard.,Feathers are always soft.,Feathers come from animals.,Feathers have different shapes,Feathers have different shapes,"Feathers are soft, as people want pillows to b..."
3,Straight razors are great for trimming beards,Straight razors are great for cutting sheet metal,The blades of Straight razors are too thin to ...,Straight razors are made in metal,sheet metal is lighter than Straight razors,sheet metal is lighter than Straight razors,Straight razors are only used for shaving and ...
4,You are likely to find an orange tree in Florida.,You are likely to find an orange tree in Antar...,It is too cold for an orange tree to grow in A...,People can take oranges to Antarctica for eating,That pile of snow in Antarctica seems like an ...,That pile of snow in Antarctica seems like an ...,It is too cold in Antarctica for an orange tre...
...,...,...,...,...,...,...,...
995,Heat energy can convert to light energy.,Lamps can convert peanut butter for energy.,Peanut butter is not directly convertible to l...,Peanuts are energy food for people but not light,the lamp looks like a peanut,the lamp looks like a peanut,Peanut butter is an eatable and cannot be conv...
996,He ride a bicycle on road,He ride a bicycle on water,water cant not support the bicycle,water is for drinking but bicycle is for riding,There is an image of water on the bicycle,There is an image of water on the bicycle,Bicycles can not be ridden on water
997,The boy wore uniform to go to school,The boy wore uniform to go to restaurant,People do not wear uniform for restaurant,Restaurant is always crowded,Restaurants don't sell uniforms,Restaurants don't sell uniforms,Restaurants do not require uniform for its cus...
998,my brother is driving a car,my giraffe is driving a car,giraffes do not know how to drive a car,giraffes are usually brown while cars are usua...,a giraffe can not eat a car,a giraffe can not eat a car,"Giraffes could not fit in a car, nor could the..."


In [22]:
corr_statement= []
incorr_statement=[]
right_reason1= []
right_reason2 = []

for i in range(len(train_data['Correct Statement'])):
  correct_statement.append([train_data.iloc[i,0], 1])
  incorrect_statement.append([train_data.iloc[i,1], 0])
  right_reason1.append([train_data.iloc[i,2], 1])
  right_reason2.append([train_data.iloc[i,3], 1])

for i in range(len(test_data['Correct Statement'])):
  correct_statement.append([test_data.iloc[i,0], 1])
  incorr_statement.append([test_data.iloc[i,1], 0])
  right_reason1.append([test_data.iloc[i,2], 1])
  right_reason2.append([test_data.iloc[i,3], 1])

for i in range(len(valid_data['Correct Statement'])):
  corr_statement.append([valid_data.iloc[i,0], 1])
  incorrect_statement.append([valid_data.iloc[i,1], 0])
  right_reason1.append([valid_data.iloc[i,2], 1])
  right_reason2.append([valid_data.iloc[i,3], 1])


In [23]:
data_list = corr_statement + incorr_statement + right_reason1 + right_reason2
random.shuffle(data_list) 
data_list

[["stone isn't a kind of food", 1],
 ['Eating apples is good', 1],
 ['Developed countries are relatively advanced', 1],
 ['The sand found in the sahara desert can cure cancer in Japan', 1],
 ["Fish can't fly, they swim.", 1],
 ['A car can be different colors and the moon is all gray.', 1],
 ['sugar usually is used as a sweetener', 1],
 ['My father likes doing strength training before going to park', 1],
 ['Ostrich is very fast, and he is also the highest bird in the world.', 1],
 ['my android mobile installs games from the app store', 0],
 ['Coins are usually metal whereas birds have feathers.', 1],
 ['A bananas is generally yellow while a phone can be many different colors.',
  1],
 ["Students test teachers' patience.", 1],
 ['Homework is too many and a sewer is dirty', 1],
 ["A motor vehicle doesn't need a car window.", 1],
 ['animals are afraid of fire.', 1],
 ['Bowling balls are heavy and sink.', 1],
 ["America doesn't border Japan", 1],
 ["Luggage can't move by themselves", 1],
 [

In [24]:
test_list = corr_statement + incorr_statement + right_reason1 + right_reason2
random.shuffle(test_list) 
test_list

[['Chairs have four legs', 1],
 ['The light of the moon is the result of reflecting sunlight', 1],
 ['Cat food and human food contain similar nutrients', 1],
 ["human can't live without water", 1],
 ['The monkey is much larger than a cup.', 1],
 ['Zebras do not hunt lions.', 1],
 ['only a qualified surgeon can perform a surgery', 1],
 ["even if we can't swim and jump to save people, someone will save us", 1],
 ['monkeys cannot fly, birds can fly', 1],
 ['most people in France speak French to others', 1],
 ['Birds are good at preying fishes which are good at swimming', 1],
 ['The man walked to the store.', 1],
 ['taking a deep breath in advance is helpful to stay longer underwater', 1],
 ['Ants prefer to eat ice cream.', 1],
 ['People need to be social to make friends and get information', 1],
 ['The wheel was invented by pre-historic humans', 1],
 ['Dumbells do not get along with pickles.', 1],
 ['Sleeping with makeup on is bad for your skin, it can make you break out.',
  1],
 ['Somet

In [25]:
validation_list = corr_statement + incorr_statement + right_reason1 + right_reason2
random.shuffle(validation_list) 
validation_list

[['It takes years to learn how to play chess', 1],
 ['A horse is much larger than a wallet.', 1],
 ['Rocks are inedible.', 1],
 ['A book and a clock is very essential in our daily life', 1],
 ['there are plays in the theater not books.', 1],
 ['A computer is an inanimate object', 1],
 ['Usually lions live in forests or zoo', 1],
 ["The rice doesn't taste good enough.", 1],
 ['a ceiling fan has no modules to light.', 1],
 ['Water cannot attract usually magnets attract', 1],
 ["Windows doesn't look at people", 1],
 ['an older version of computer hardware is usually less efficient', 1],
 ['The sun is too big to lay under the dog.', 1],
 ['the burned river can still not prevent his army from escaping', 1],
 ['Dogs are not able to jump between tree branches', 1],
 ['People dress up in lots of costumes besides just ghosts for Halloween', 1],
 ['pen is an object to write', 1],
 ["barber's pet is sick", 1],
 ['backyard is too big for people to store their toothbrush.', 1],
 ['Round wheels are 

In [26]:
column_labels = ['Sentence', 'Labels']
train_data_df = pd.DataFrame(data_list, columns = column_labels)
test_data_df = pd.DataFrame(test_list, columns = column_labels)
dev_data_df = pd.DataFrame(validation_list, columns = column_labels)

In [27]:
train_data_df.head()

,Sentence,Labels
0,stone isn't a kind of food,1
1,Eating apples is good,1
2,Developed countries are relatively advanced,1
3,The sand found in the sahara desert can cure c...,1
4,"Fish can't fly, they swim.",1


In [28]:
(X_train, Y_train) , (X_test, Y_test), preprocess = text.texts_from_df(train_df= train_data_df, 
                                                                       text_column= 'Sentence', 
                                                                       label_columns= 'Labels',
                                                                       val_df = dev_data_df,
                                                                       maxlen = 400,
                                                                       preprocess_mode = 'bert')      

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [29]:
X_train[0].shape

(26000, 400)

In [30]:
model = text.text_classifier(name='bert', 
                             train_data = (X_train, Y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [31]:
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, Y_train),
                             val_data = (X_test, Y_test),
                             batch_size = 6)

In [32]:
learner.fit_onecycle(lr = 2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4334/4334 [==============================] - 3198s 738ms/step - loss: 0.1353 - accuracy: 0.9598 - val_loss: 0.1042 - val_accuracy: 0.9615


In [33]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [34]:
predictor.save('/content/drive/My Drive/bert')

In [35]:
result = predictor.predict(list(test_data_df['Sentence']))
result

['Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',
 'Labels',

In [36]:
for i in range(len(result)):
  if result[i] == 'Labels':
    result[i] = 1
  else:
    result[i] = 0

In [37]:
result

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [38]:
test_data_df

,Sentence,Labels
0,Chairs have four legs,1
1,The light of the moon is the result of reflect...,1
2,Cat food and human food contain similar nutrients,1
3,human can't live without water,1
4,The monkey is much larger than a cup.,1
...,...,...
25995,Crying can show people happy and happy,1
25996,Apple is a technology company that sells perso...,1
25997,The snake sleeps on the mice that come through...,0
25998,bike has two tires and square has four sides,1


In [39]:
count=0
for i in range (len(result)):
  if result[i] == test_data_df.iloc[i,1]:
    count+=1
accuracy_test = (count/len(result)) * 100
print("Accuracy on Test Dataset: ", accuracy_test, "%")

Accuracy on Test Dataset:  96.15384615384616 %


In [40]:
predictor_load = ktrain.load_predictor('/content/drive/MyDrive/bert')

In [42]:
t_f = '/content/drive/MyDrive/assignmnet data/SemEval2020-Task4-Commonsense-Validation-and-Explanation/ALL data/Test Data/subtaskA_test_data.csv'
test_data = pd.read_csv(t_f)

In [43]:
result = predictor.predict(list(test_data['sent1']))
for i in range(len(result)):
  if result[i] == 'Labels':
    result[i] = 1
  else:
    result[i] = 0
result 

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [44]:
len(result)

1000

In [45]:
t_ids = list(test_data['id'])
len(t_ids)

1000

In [46]:
t_list = []
for i in range(len(result)):
  t_list.append([t_ids[i],result[i]])
t_list

[[1175, 1],
 [452, 1],
 [275, 1],
 [869, 1],
 [50, 1],
 [1155, 1],
 [1395, 1],
 [967, 1],
 [906, 1],
 [1465, 1],
 [1680, 1],
 [1964, 1],
 [830, 1],
 [174, 1],
 [461, 1],
 [1978, 1],
 [495, 1],
 [1388, 1],
 [1572, 1],
 [305, 1],
 [1695, 1],
 [1886, 1],
 [878, 1],
 [754, 1],
 [1444, 1],
 [1065, 1],
 [543, 1],
 [518, 1],
 [692, 1],
 [1296, 1],
 [1893, 1],
 [1915, 1],
 [418, 1],
 [953, 1],
 [1148, 1],
 [1943, 1],
 [105, 1],
 [1222, 1],
 [400, 1],
 [1610, 1],
 [230, 1],
 [1172, 1],
 [947, 1],
 [1144, 1],
 [1017, 1],
 [1063, 1],
 [930, 1],
 [83, 1],
 [1145, 1],
 [363, 1],
 [338, 1],
 [391, 1],
 [1139, 1],
 [688, 1],
 [704, 1],
 [1544, 1],
 [1760, 1],
 [303, 1],
 [850, 1],
 [924, 1],
 [1840, 1],
 [292, 1],
 [7, 1],
 [800, 1],
 [1177, 1],
 [259, 1],
 [274, 1],
 [1031, 1],
 [377, 1],
 [1225, 1],
 [396, 1],
 [1860, 1],
 [1500, 1],
 [1625, 1],
 [896, 1],
 [1704, 1],
 [1280, 1],
 [125, 1],
 [1509, 1],
 [458, 1],
 [697, 1],
 [1483, 1],
 [317, 1],
 [757, 1],
 [770, 1],
 [1976, 1],
 [970, 1],
 [1712,

In [47]:
test_data = pd.DataFrame(t_list)

In [48]:
gfg_csv_data = test_data.to_csv('subtaskA_answers.csv', index = True) 